# MLP regressor to find a math function

This is a very simple testing of a [Multi-layer Perceptron Neural Network](https://en.wikipedia.org/wiki/Multilayer_perceptron). The objective is to teach an algorithm how to solve a function giving it just the values and results.

In [1]:
import numpy as np
import pandas as pd
from sklearn import neural_network
from sklearn import model_selection

We will teach the computer how to solve the next function:

$f(x, y, z) = x^{2} + yz$ ; where $0 <= x,y,z <= 9$

without giving it the explicit function.

In [2]:
# The function to be solved
def complex_function(x, y, z):
    return x + y * z

This function will create a dataset with random input variables `[a, b, c]`. The values contained in each variable will only be on the ranges from 0-99. A result column `f` will be attached to the dataset. This function will be useful later to create a small dataset to check how the model performed.

In [3]:
def create_dataset(df_size=100):
    dataset = pd.DataFrame(
        np.random.randint(9, size=(df_size, 3)),
        columns=['a', 'b', 'c'])
    dataset['f'] = dataset.apply(
        lambda row: complex_function(row['a'], row['b'], row['c']),
        axis=1)
    return dataset

We will create **two million samples**, 20% of which will be used for validation (so, *only* about 1.8 million samples will be used for training). The column `f` will store the result of the `complex_function`.

In [4]:
# Split the dataset for training and validation
df = create_dataset(2000000)
x_train, x_test, y_train, y_test = model_selection.train_test_split(
    df[['a', 'b', 'c']],
    df[['f']],
    test_size=0.20)
df.head()

,a,b,c,f
0,3,6,4,27
1,5,1,3,8
2,2,1,2,4
3,8,8,3,32
4,3,0,7,3


Let's create the actual model. The `MLPRegressor` model is a very simple feed forward network very useful for regression analysis. In this case, it has two hidden layer with nine and three inputs.

In [5]:
clf = neural_network.MLPRegressor(
    solver='lbfgs',
    alpha=1e-5,
    hidden_layer_sizes=(9, 9),
    verbose=True)

print("Training...")
clf.fit(x_train.values, y_train.values.ravel())
print("Training finished")

print("Predicting...")
pred = clf.predict(x_test.values)
print("Prediction finished")

clf_score = clf.score(x_test, y_test)
print("Accuracy score: {}".format(clf_score))

Training...
Training finished
Predicting...
Prediction finished
Accuracy score: 0.9988890283277395


We have a very *acceptable* score ($99.888\% $ accuracy score), so let's find out how it's behaving with a small number of new samples.

In [8]:
validation_data = create_dataset(10)
# Create a column with results taken from the neural net
validation_data['predicted'] = np.array(clf.predict(validation_data[['a', 'b', 'c']].values))
# Compute the difference between the predicted and actual results
validation_data['diff'] = validation_data['predicted'] - validation_data['f']

validation_data

,a,b,c,f,predicted,diff
0,3,4,1,7,7.296939,0.296939
1,8,2,6,20,20.463242,0.463242
2,3,4,1,7,7.296939,0.296939
3,7,5,8,47,46.298738,-0.701262
4,0,8,0,0,0.686814,0.686814
5,1,4,2,9,9.792165,0.792165
6,4,8,4,36,36.084938,0.084938
7,0,7,0,0,0.447795,0.447795
8,6,3,0,6,5.029361,-0.970639
9,0,1,8,8,7.425841,-0.574159


## Conclusion
As you can see, we were able to achieve pretty much the same results as the original function. This demo didn't use *as many* samples and variaty of data as I would have liked. But you can change the `df_size` parameter to something like ten millions and the range of the numbers from [0, 9] to [0, 99] in order to achieve better results (note that the computational resources neede will be quite larger that the ones used to train this network).